In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

from dotenv import load_dotenv

In [30]:
# Установка максимального количества выводимых строк в DataFrame
pd.set_option('display.max_rows', None)

In [43]:
load_dotenv()

False

# Загрузка и чтение файлов в формате txt

In [ ]:
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Получаем текущий рабочий каталог
current_dir = os.getcwd()

# Определяем путь к файлу
file_path = os.path.join(current_dir, "/Users/sergey/Desktop/RAG_Project/RAG_intelion/Intelion_books/bazaznanii.txt")

# Определяем путь к директории для хранилища
persistent_directory = os.path.join(current_dir, "db", "chroma_db_bazaznanii")


db_dir = os.path.join(current_dir, "db")

# Проверка, существует ли текстовый файл, с которого нужно загружать данные
if not os.path.exists(file_path):
    raise FileNotFoundError(f"Файл {file_path} не найден. Проверьте путь.")
else:
    # Если файл найден, загружаем его содержимое с помощью загрузчика текстовых данных
    loader = TextLoader(file_path)
    documents = loader.load()  # Загружаем документы из текстового файла
    print(f"Загружено {len(documents)} документов.")

# Если документы загружены, используем рекурсивный сплиттер для разбиения
print("\n--- Используем рекурсивное разбиение на чанки ---")
Recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100, chunk_overlap=30
)

# Разбиваем документы на чанки
docs = Recursive_text_splitter.split_documents(documents)

print(f"Разбито на {len(docs)} чанков.")

In [92]:
# Объединение пути к текущему каталогу с подкаталогом "db" и именем каталога "chroma_db"
# Это создает полный путь к каталогу для хранения данных "chroma_db"
# persistent_directory = os.path.join(current_dir, "db", "chroma_db_intelion_PDF")

# db_dir = os.path.join(current_dir, "db")

# В качестве эмбеддингов используем text-embedding-3-small от OpenAI

In [44]:
# Загружаем модель эмбеддингов
embeddings = OpenAIEmbeddings(model="text-embedding-3-small",)

In [7]:
# Функция для создания и сохранения векторного хранилища
def create_vector_store(docs, store_name):
    
    # Определение пути к директории, где будет сохранено векторное хранилище
    persistent_directory = os.path.join(db_dir, store_name)
    
    # Проверка, существует ли уже директория для хранения векторного хранилища
    if not os.path.exists(persistent_directory):
        
        # Если директория не существует, выводим сообщение о создании векторного хранилища
        print(f"\n--- Создание векторного хранилища {store_name} ---")
        
        # Создаем векторное хранилище из документов с использованием заданной модели embeddings
        # и сохраняем его в указанную директорию
        db = Chroma.from_documents(
            docs, embeddings, persist_directory=persistent_directory
        )
        
        # Сообщаем, что создание векторного хранилища завершено
        print(f"--- Создание векторного хранилища завершено {store_name} ---")
    
    # Если директория уже существует, выводим сообщение, что хранилище уже создано, и инициализация не требуется
    else:
        print(
            f"Хранилище {store_name} хранилище уже создано, и инициализация не требуется.")



# Создаем векторную базу данных

In [ ]:

create_vector_store(docs, "chroma_db_intelion_bazaznanii")


# Загрузка векторной базы данных после ее создания или если она была создана ранее

In [45]:
import os
# from chromadb import Chroma  # Импортируем библиотеку Chroma

# Укажите путь к вашей базе данных
persistent_directory = "/Users/sergey/Desktop/RAG_Project/RAG_intelion/db/chroma_db_intelion_bazaznanii"

# Проверьте, существует ли база данных
if os.path.exists(os.path.join(persistent_directory, "chroma.sqlite3")):
    print(f"Загружаем векторное хранилище из {persistent_directory}...")

    # Загружаем векторное хранилище
    db = Chroma(
        persist_directory=persistent_directory,  # Указываем путь к существующему хранилищу
        embedding_function=embeddings  # Передаем функцию эмбеддингов (нужна для поиска)
    )

In [46]:
from langchain_core.messages import HumanMessage, SystemMessage

# Проверяем как работает поиск по базе данных

In [47]:
# Определяем вопрос, на который будем искать ответ
query = "Кто является генеральным директором?"

# 1. Поиск по сходству без порога (search_type="similarity")
retriever_similarity = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)

# Выполняем поиск
relevant_docs_similarity = retriever_similarity.invoke(query)

# Вывод результатов
print("\n--- Результаты поиска по сходству (без порога) ---")
for i, doc in enumerate(relevant_docs_similarity, 1):
    print(f"Документ {i}:\n{doc.page_content}\n")

# 2. Поиск с использованием функции search с порогом сходства
def search(query: str, db, k=3, similarity_threshold=0.4):
    # Поиск документов с оценками релевантности
    docs_scores = db.similarity_search_with_relevance_scores(query, k=k)
    # Фильтрация по порогу сходства
    good_documents = []
    for doc, score in docs_scores:
        if score > similarity_threshold:
            good_documents.append(doc.page_content)
    return good_documents

# Выполняем поиск с порогом
results_with_threshold = search(query, db=db, k=3, similarity_threshold=0.2)

# Вывод результатов
print("\n--- Результаты поиска с порогом сходства ---")
for i, content in enumerate(results_with_threshold, 3):
    print(f"Документ {i}:\n{content}\n")



--- Результаты поиска по сходству (без порога) ---
Документ 1:
ЕС – Екатерина Евгеньевна Семыкина, Заместитель генерального директора

Документ 2:
ТА – Тимофей Андреевич Семенов, Генеральный директор

Документ 3:
поздразделения, то обращаться нужно напрямую к генеральному директору или к HRD Анастасии Нурмамед.


--- Результаты поиска с порогом сходства ---
Документ 3:
ЕС – Екатерина Евгеньевна Семыкина, Заместитель генерального директора

Документ 4:
ТА – Тимофей Андреевич Семенов, Генеральный директор

Документ 5:
поздразделения, то обращаться нужно напрямую к генеральному директору или к HRD Анастасии Нурмамед.



# Здесь добовляем генерацию ответа LLM

In [48]:
# Определяем вопрос, на который будем искать ответ
query = "Кто является генеральным директором?"

# Получаем релевантные документы на основе запроса
retriever = db.as_retriever(
    search_type="similarity",  # Указываем тип поиска по сходству
    search_kwargs={"k": 5},  # Параметр: количество возвращаемых документов (k)
)

# Выполняем запрос к ретриверу для получения релевантных документов
relevant_docs = retriever.invoke(query)

# Выводим релевантные документы с их содержимым
print("\n--- Релевантные документы ---")
for i, doc in enumerate(relevant_docs, 1):
    # Выводим каждый документ с его содержимым и номером
    print(f"Документы {i}:\n{doc.page_content}\n")

# Объединяем запрос и содержимое релевантных документов в один текст для передачи модели
combined_input = (
    "Это некоторые документы которые могут помочь ответить на вопрос: "
    + query  # Включаем исходный запрос
    + "\n\nРелевантные документы:\n"  # Добавляем заголовок для раздела с документами
    + "\n\n".join([doc.page_content for doc in relevant_docs])  # Добавляем текст всех документов
    + "\n\nПожалуйста, предоставьте ответ, основываясь только на предоставленных документах. Если ответ не найден в документах, ответьте ‘Я не уверен'."  # Просим модель ответить только на основе предоставленных данных
)


# Создаем модель ChatOpenAI для обработки запроса
model = ChatOpenAI(model="gpt-4o-mini")

# Определяем сообщения, которые передаем модели
messages = [
    SystemMessage(content="Ты помошник который отвечает на вопросы пользователя."),  # Системное сообщение с инструкциями для модели
    HumanMessage(content=combined_input),  # Сообщение пользователя, содержащее запрос и документы
]

# Вызываем модель с подготовленными сообщениями
result = model.invoke(messages)

# Выводим сгенерированный ответ
print("\n--- Сгенерированный ответ ---")
# print("Full result:")  # Можно также вывести полный результат, если нужно
# print(result)
print("Текстовый ответ модели:")  # Выводим только текстовый ответ модели
print(result.content)


--- Релевантные документы ---
Документы 1:
ЕС – Екатерина Евгеньевна Семыкина, Заместитель генерального директора

Документы 2:
ТА – Тимофей Андреевич Семенов, Генеральный директор

Документы 3:
поздразделения, то обращаться нужно напрямую к генеральному директору или к HRD Анастасии Нурмамед.

Документы 4:
АР – Александр Александрович Рощин, Операционный директор

Документы 5:
Операционный директор Александр Рощин


--- Сгенерированный ответ ---
Текстовый ответ модели:
Генеральным директором является Тимофей Андреевич Семенов.


In [49]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Вариант с выводом перефразированных вопросов

In [ ]:
# Инициализируем retriever с типом поиска по схожести, чтобы получать k=3 наиболее релевантных результатов
retriever = db.as_retriever(
    search_type="similarity",  # Используем метод поиска по схожести
    search_kwargs={"k": 5},  # Количество возвращаемых результатов: 3
)

# Создаем модель ChatOpenAI с использованием модели GPT-4o
llm = ChatOpenAI(model="gpt-4o-mini")  # Модель GPT-4o для обработки запросов

# Определяем системный промпт для контекстуализации вопросов
# Этот промпт объясняет ИИ, как следует формулировать вопрос на основе истории чата
contextualize_q_system_prompt = (
    "Учитывая историю чата и последний вопрос пользователя, "  # Описание задачи
    "который может ссылаться на контекст из истории чата, "  # Учитывание истории чата
    "сформулируйте самодостаточный вопрос, который можно понять "  # Требование сделать вопрос самодостаточным
    "без использования истории чата. НЕ отвечайте на вопрос, просто "  # Не нужно отвечать, только переформулировать вопрос
    "переформулируйте его, если это необходимо, или верните его без изменений."
# )
    # "Вы являетесь ассистентом по языковой модели искусственного интеллекта."
    # "Ваша задача - сгенерировать 3 различных подвопросов Или альтернативных версий данного пользовательского вопроса для извлечения соответствующих документов из векторной базы данных."
    # "Сгенерировав несколько версий пользовательского вопроса, ваша цель - помочь пользователю преодолеть некоторые ограничения поиска сходства на основе расстояния."
    # "Создавая дополнительные вопросы, вы можете разбить вопросы, относящиеся к нескольким концепциям, на отдельные вопросы."
    # "Это поможет вам получить соответствующие документы для составления окончательного ответа"
    # "Если в вопросе присутствует несколько концепций, вам следует разбить его на подзадачи, задав по одному вопросу для каждой концепции"
    )

# Создаем шаблон для формирования контекстуализированных вопросов
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),  # Системное сообщение с инструкцией для ИИ
        MessagesPlaceholder("chat_history"),  # Место для истории чата
        ("human", "{input}"),  # Ввод пользователя, который нужно переформулировать
    ]
)

# Создаем retriever, учитывающий историю чата
# Этот retriever использует LLM для помощи в переформулировании вопросов с учетом контекста
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt  # Используем созданный шаблон и retriever для поиска с историей
)

# Определяем системный промпт для ответа на вопросы
# Этот промпт объясняет ИИ, как отвечать на вопросы на основе контекста и что делать, если ответа нет
qa_system_prompt = (
    "Вы помощник для задач по ответам на вопросы о людях из русской Wikipedia. Используйте "  # Инструкция для ИИ
    "следующие части полученного контекста, чтобы ответить на "  # Использование контекста для ответа
    "вопрос. Если вы не знаете ответа, просто скажите, что вы "  # Если ответа нет, сообщить об этом
    "не знаете. Используйте максимум три предложения и дайте "  # Ответ должен быть кратким
    "краткий ответ."  # Старайтесь отвечать как можно более сжато
    "\n\n"
    "{context}"  # Контекст, который будет использован для ответа
)

# Создаем шаблон для формирования ответов на вопросы
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),  # Системное сообщение с инструкцией для ИИ
        MessagesPlaceholder("chat_history"),  # Место для истории чата
        ("human", "{input}"),  # Ввод пользователя, на который нужно ответить
    ]
)

# Создаем цепочку для объединения документов для ответов на вопросы
# `create_stuff_documents_chain` передает весь найденный контекст в LLM для обработки
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Создаем retrieval-цепочку, которая объединяет retriever с историей и цепочку для ответов на вопросы
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


# # Функция для моделирования непрерывного чата
# def continual_chat():
#     print("Начните общение с ИИ! Введите ‘выход’, чтобы завершить разговор.")  # Начало общения с ИИ
#     chat_history = []  # Хранение истории чата
#     while True:
#         query = input("ВЫ: ")  # Получение запроса от пользователя
#         if query.lower() == "выход":  # Если пользователь хочет завершить чат
#             break
#         # Обработка запроса пользователя через цепочку retrieval
#         result = rag_chain.invoke({"input": query, "chat_history": chat_history})  # Запрос к цепочке
#         # Печать вопроса пользователя и ответа ИИ
#         print(f"Вы: {query}")  # Печать вопроса пользователя
#         print(f"AI: {result['answer']}")  # Ответ ИИ
#         # Обновление истории чата
#         chat_history.append(HumanMessage(content=query))  # Добавление сообщения пользователя в историю
#         chat_history.append(SystemMessage(content=result["answer"]))  # Добавление ответа ИИ в историю

# # Main function to start the continual chat
# if __name__ == "__main__":
#     continual_chat()


# Функция для моделирования непрерывного чата с выводом переформулированного вопроса
def continual_chat():
    print("Начните общение с ИИ помошником о компании Intellion! Введите ‘выход’, чтобы завершить разговор.")  # Начало общения с ИИ
    chat_history = []  # Хранение истории чата
    while True:
        query = input("ВЫ: ")  # Получение запроса от пользователя
        if query.lower() == "выход":  # Если пользователь хочет завершить чат
            break

        # Переформулировка вопроса через вызов LLM с контекстом
        reformulated_prompt = contextualize_q_prompt.format_messages(
            chat_history=chat_history, input=query
        )
        reformulated_query_result = llm(reformulated_prompt)  # Получаем объект AIMessage
        reformulated_query = reformulated_query_result.content  # Доступ к содержимому через атрибут content
        
        print(f"Переформулированный вопрос: {reformulated_query}")  # Вывод переформулированного вопроса
        
        # Обработка запроса пользователя через цепочку retrieval
        result = rag_chain.invoke({"input": reformulated_query, "chat_history": chat_history})  # Запрос к цепочке
        
        # Печать вопроса пользователя и ответа ИИ
        print(f"Вы: {query}")  # Печать вопроса пользователя
        print(f"AI: {result['answer']}")  # Ответ ИИ
        
        # Обновление истории чата
        chat_history.append(HumanMessage(content=query))  # Добавление сообщения пользователя в историю
        chat_history.append(SystemMessage(content=result["answer"]))  # Добавление ответа ИИ в историю

# Main function to start the continual chat
if __name__ == "__main__":
    continual_chat()

In [39]:
from langchain_core.tools import Tool
from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent, create_react_agent
from langchain.memory import ConversationBufferMemory
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_core.messages import AIMessage, HumanMessage

Respond to the human as helpfully and accurately as possible. You have access to the following tools:

{tools}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per $JSON_BLOB, as shown:

```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}

Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation

# Здесь я использовал  ReAct Docstore для взаимодействия с пользователем (как он выглядит видно выше)

## Результат хороший но используется доп API

In [ ]:
# Загружаем существующее хранилище векторов с функцией эмбеддинга
db = Chroma(persist_directory=persistent_directory,
            embedding_function=embeddings)  # Инициализируем хранилище векторов (Chroma) с указанием директории для хранения и функции эмбеддинга

# Создаем retriever для поиска по хранилищу векторов
# `search_type` указывает тип поиска (например, по схожести)
# `search_kwargs` содержит дополнительные параметры поиска (например, количество возвращаемых результатов)
retriever = db.as_retriever(
    search_type="similarity",  # Указываем тип поиска — по схожести
    search_kwargs={"k": 3},  # Возвращаем 3 наиболее релевантных результата
)

# Создаем модель ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=os.getenv("OPENAI_API_KEY"))   # Используем модель GPT-4o для работы с текстовыми данными

# Определяем системный промпт для контекстуализации вопросов
# Этот промпт помогает ИИ понять, что необходимо переформулировать вопрос
# на основе истории чата, чтобы сделать его самодостаточным
contextualize_q_system_prompt = (
    "Учитывая историю чата и последний вопрос пользователя, "
    "который может ссылаться на контекст из истории чата, "
    "сформулируйте самодостаточный вопрос, который можно понять "
    "без использования истории чата. НЕ отвечайте на вопрос, просто "
    "переформулируйте его, если это необходимо, или верните его без изменений."
)

# Создаем шаблон для контекстуализации вопросов
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),  # Системное сообщение с инструкцией
        MessagesPlaceholder("chat_history"),  # Место для подстановки истории чата
        ("human", "{input}"),  # Ввод пользователя, который нужно переформулировать
    ]
)

# Создаем retriever, учитывающий историю чата
# Этот retriever использует LLM для переформулирования вопросов на основе истории чата
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt  # Используем созданный шаблон и retriever для поиска с историей
)

# Определяем системный промпт для ответа на вопросы
# Этот промпт помогает ИИ давать краткие ответы, основываясь на полученном контексте
# и указывает, что делать, если ответа нет
qa_system_prompt = (
    "Вы помощник для задач по ответам на вопросы о компании Intelion. Используйте "
    "следующие части полученного контекста, чтобы ответить на "
    "вопрос. Если вы не знаете ответа, просто скажите, что вы "
    "не знаете. Используйте максимум три предложения и дайте "
    "краткий ответ."
    "\n\n"
    "{context}"  # Подстановка контекста для ответа
)

# Создаем шаблон для формирования ответов на вопросы
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),  # Системное сообщение с инструкциями для ответа
        MessagesPlaceholder("chat_history"),  # История чата
        ("human", "{input}"),  # Ввод пользователя
    ]
)

# Создаем цепочку для объединения документов для ответов на вопросы
# `create_stuff_documents_chain` передает весь полученный контекст в LLM для обработки
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Создаем retrieval-цепочку, которая объединяет retriever с историей и цепочку для ответов на вопросы
rag_chain = create_retrieval_chain(
    history_aware_retriever, question_answer_chain  # Комбинируем поиск и цепочку ответа на вопросы
)

# Настройка ReAct-агента с использованием хранилища документов для поиска
# Загружаем промпт ReAct Docstore для взаимодействия с пользователем через документы
react_docstore_prompt = hub.pull("hwchase17/react", api_key=os.getenv("LANGCHAIN_API_KEY"))  # Промпт для ReAct-агента

# Определяем инструменты, которые агент может использовать
tools = [
    Tool(
        name="Answer Question",  # Название инструмента
        func=lambda input, **kwargs: rag_chain.invoke(
            {"input": input, "chat_history": kwargs.get("chat_history", [])}
        ),  # Вызов функции для получения ответа с использованием цепочки поиска и истории
        description="Полезно для ответов на вопросы по контексту.",  # Описание инструмента
    )
]

# Создаем ReAct-агента с использованием хранилища документов
agent = create_react_agent(
    llm=llm,  # Языковая модель
    tools=tools,  # Инструменты, доступные агенту
    prompt=react_docstore_prompt,  # Промпт для взаимодействия
)

# Инициализируем AgentExecutor для управления взаимодействием между пользователем, агентом и инструментами
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, handle_parsing_errors=True, verbose=True,  # Обработка ошибок и вывод процесса
)

# Инициализируем историю чата
chat_history = []
while True:
    query = input("Вы: ")  # Получаем ввод пользователя
    if query.lower() == "выход":  # Если введен 'exit', завершаем цикл
        break
    response = agent_executor.invoke(
        {"input": query, "chat_history": chat_history})  # Агент обрабатывает запрос с учетом истории чата
    print(f"ИИ: {response['output']}")  # Выводим ответ агента

    # Обновляем историю чата
    chat_history.append(HumanMessage(content=query))  # Добавляем сообщение пользователя в историю
    chat_history.append(AIMessage(content=response["output"]))  # Добавляем ответ агента в историю

In [21]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser

# Самый быстрый и в целом четкийй вариант

In [27]:
# Инициализируем retriever с типом поиска по схожести, чтобы получать k=3 наиболее релевантных результатов
retriever = db.as_retriever(
    search_type="similarity",  # Используем метод поиска по схожести
    search_kwargs={"k": 5},  # Количество возвращаемых результатов: 3
)

# Создаем модель ChatOpenAI с использованием модели GPT-4o
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)  # Модель GPT-4o для обработки запросов

# Определяем системный промпт для контекстуализации вопросов
# Этот промпт объясняет ИИ, как следует формулировать вопрос на основе истории чата
contextualize_q_system_prompt = (
    "Учитывая историю чата и последний вопрос пользователя, "  # Описание задачи
    "который может ссылаться на контекст из истории чата, "  # Учитывание истории чата
    "сформулируйте самодостаточный вопрос, который можно понять "  # Требование сделать вопрос самодостаточным
    "без использования истории чата. НЕ отвечайте на вопрос, просто "  # Не нужно отвечать, только переформулировать вопрос
    "переформулируйте его, если это необходимо, или верните его без изменений."
# )
    # "Вы являетесь ассистентом по языковой модели искусственного интеллекта."
    # "Ваша задача - сгенерировать 3 различных подвопросов Или альтернативных версий данного пользовательского вопроса для извлечения соответствующих документов из векторной базы данных."
    # "Сгенерировав несколько версий пользовательского вопроса, ваша цель - помочь пользователю преодолеть некоторые ограничения поиска сходства на основе расстояния."
    # "Создавая дополнительные вопросы, вы можете разбить вопросы, относящиеся к нескольким концепциям, на отдельные вопросы."
    # "Это поможет вам получить соответствующие документы для составления окончательного ответа"
    # "Если в вопросе присутствует несколько концепций, вам следует разбить его на подзадачи, задав по одному вопросу для каждой концепции"
    )

# Создаем шаблон для формирования контекстуализированных вопросов
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),  # Системное сообщение с инструкцией для ИИ
        MessagesPlaceholder("chat_history"),  # Место для истории чата
        ("human", "{input}"),  # Ввод пользователя, который нужно переформулировать
    ]
)

# Создаем retriever, учитывающий историю чата
# Этот retriever использует LLM для помощи в переформулировании вопросов с учетом контекста
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt  # Используем созданный шаблон и retriever для поиска с историей
)

# Определяем системный промпт для ответа на вопросы
# Этот промпт объясняет ИИ, как отвечать на вопросы на основе контекста и что делать, если ответа нет
qa_system_prompt = (
    "Вы помощник для задач по ответам на вопросы о компании Intelion. Используйте "  # Инструкция для ИИ
    "следующие части полученного контекста, чтобы ответить на "  # Использование контекста для ответа
    "вопрос. Если вы не знаете ответа, просто скажите, что вы "  # Если ответа нет, сообщить об этом
    "не знаете. Используйте максимум три предложения и дайте "  # Ответ должен быть кратким
    "краткий ответ."  # Старайтесь отвечать как можно более сжато
    "\n\n"
    "{context}"  # Контекст, который будет использован для ответа
)

# Создаем шаблон для формирования ответов на вопросы
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),  # Системное сообщение с инструкцией для ИИ
        MessagesPlaceholder("chat_history"),  # Место для истории чата
        ("human", "{input}"),  # Ввод пользователя, на который нужно ответить
    ]
)

# Создаем цепочку для объединения документов для ответов на вопросы
# `create_stuff_documents_chain` передает весь найденный контекст в LLM для обработки
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Создаем retrieval-цепочку, которая объединяет retriever с историей и цепочку для ответов на вопросы
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


# Функция для моделирования непрерывного чата
def continual_chat():
    print("Начните общение с ИИ! Введите ‘выход’, чтобы завершить разговор.")  # Начало общения с ИИ
    chat_history = []  # Хранение истории чата
    while True:
        query = input("ВЫ: ")  # Получение запроса от пользователя
        if query.lower() == "выход":  # Если пользователь хочет завершить чат
            break
        # Обработка запроса пользователя через цепочку retrieval
        result = rag_chain.invoke({"input": query, "chat_history": chat_history})  # Запрос к цепочке
        # Печать вопроса пользователя и ответа ИИ
        print(f"Вы: {query}")  # Печать вопроса пользователя
        print(f"AI: {result['answer']}")  # Ответ ИИ
        # Обновление истории чата
        chat_history.append(HumanMessage(content=query))  # Добавление сообщения пользователя в историю
        chat_history.append(SystemMessage(content=result["answer"]))  # Добавление ответа ИИ в историю

# Main function to start the continual chat
if __name__ == "__main__":
    continual_chat()


# # Функция для моделирования непрерывного чата с выводом переформулированного вопроса
# def continual_chat():
#     print("Начните общение с ИИ! Введите ‘выход’, чтобы завершить разговор.")  # Начало общения с ИИ
#     chat_history = []  # Хранение истории чата
#     while True:
#         query = input("ВЫ: ")  # Получение запроса от пользователя
#         if query.lower() == "выход":  # Если пользователь хочет завершить чат
#             break

#         # Переформулировка вопроса через вызов LLM с контекстом
#         reformulated_prompt = contextualize_q_prompt.format_messages(
#             chat_history=chat_history, input=query
#         )
#         reformulated_query_result = llm(reformulated_prompt)  # Получаем объект AIMessage
#         reformulated_query = reformulated_query_result.content  # Доступ к содержимому через атрибут content
        
#         print(f"Переформулированный вопрос: {reformulated_query}")  # Вывод переформулированного вопроса
        
#         # Обработка запроса пользователя через цепочку retrieval
#         result = rag_chain.invoke({"input": reformulated_query, "chat_history": chat_history})  # Запрос к цепочке
        
#         # Печать вопроса пользователя и ответа ИИ
#         print(f"Вы: {query}")  # Печать вопроса пользователя
#         print(f"AI: {result['answer']}")  # Ответ ИИ
        
#         # Обновление истории чата
#         chat_history.append(HumanMessage(content=query))  # Добавление сообщения пользователя в историю
#         chat_history.append(SystemMessage(content=result["answer"]))  # Добавление ответа ИИ в историю

# # Main function to start the continual chat
# if __name__ == "__main__":
#     continual_chat()

Начните общение с ИИ! Введите ‘выход’, чтобы завершить разговор.
Вы: что ты умеешь?
AI: Я могу отвечать на вопросы о компании Intelion, включая информацию о ее деятельности, задачах и стратегиях. Если у вас есть конкретный вопрос, задайте его, и я постараюсь помочь.


# Поиск документов с использованием RAG-Fusion и RRF для поиска и ранжирования документов:
## После переформулировки вопрос передается в генерацию нескольких запросов, и система выполняет поиск документов для каждого из них.

# Этот вариант работает медленнее и не сказал бы что ответы лучше

In [41]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.load import dumps, loads
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

# Инициализируем retriever с типом поиска по схожести, чтобы получать k=5 наиболее релевантных результатов
retriever = db.as_retriever(
    search_type="similarity",  # Используем метод поиска по схожести
    search_kwargs={"k": 5},  # Количество возвращаемых результатов
)

# Создаем модель ChatOpenAI с использованием модели GPT-4o
llm = ChatOpenAI(model="gpt-4o-mini")  # Модель GPT-4o для обработки запросов

# Определяем системный промпт для контекстуализации вопросов
contextualize_q_system_prompt = (
    "Учитывая историю чата и последний вопрос пользователя, "
    "который может ссылаться на контекст из истории чата, "
    "сформулируйте самодостаточный вопрос, который можно понять "
    "без использования истории чата. НЕ отвечайте на вопрос, просто "
    "переформулируйте его, если это необходимо, или верните его без изменений."
)

# Создаем шаблон для формирования контекстуализированных вопросов
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),  # Системное сообщение с инструкцией для ИИ
        MessagesPlaceholder("chat_history"),  # Место для истории чата
        ("human", "{input}"),  # Ввод пользователя, который нужно переформулировать
    ]
)

# Шаблон для генерации нескольких запросов
template = """Вы являетесь полезным помощником о компании Intellion, который генерирует несколько поисковых запросов на основе одного входного запроса. \n
Сгенерируйте несколько поисковых запросов, связанных с: {question} \n
Output (4 queries):"""

prompt_rag_fusion = ChatPromptTemplate.from_template(template)

# Функция для генерации нескольких запросов
generate_queries = (
    prompt_rag_fusion 
    | ChatOpenAI(temperature=0) 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

# Функция для Reciprocal Rank Fusion
def reciprocal_rank_fusion(results: list[list], k=60):
    """Функция RRF для слияния результатов поиска из нескольких списков."""
    fused_scores = {}
    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            fused_scores[doc_str] += 1 / (rank + k)
    
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    
    return reranked_results

# Функция для создания retrieval-цепочки с использованием RAG и RRF
def create_rag_fusion_retrieval_chain(retriever, question):
    # Генерируем несколько запросов
    generated_queries = generate_queries.invoke({"question": question})
    
    # Выполняем поиск по базе данных для каждого запроса
    search_results = [retriever.get_relevant_documents(query) for query in generated_queries]
    
    # Применяем RRF для слияния результатов
    reranked_docs = reciprocal_rank_fusion(search_results)
    
    # Извлекаем только документы, без их рангов
    return [doc for doc, score in reranked_docs]

# Создаем шаблон для ответа на вопросы на основе контекста
qa_system_prompt = (
    "Вы помощник для задач по ответам на вопросы о компании Intellion. Используйте "
    "следующие части полученного контекста, чтобы ответить на "
    "вопрос. Если вы не знаете ответа, просто скажите, что вы "
    "не знаете. Используйте максимум три предложения и дайте "
    "краткий ответ.\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Функция для обработки запроса с контекстуализацией и RAG
def process_query(query, chat_history):
    # Шаг 1: Переформулируем вопрос с учетом истории чата
    reformulated_prompt = contextualize_q_prompt.format_messages(
        chat_history=chat_history, input=query
    )
    reformulated_query_result = llm(reformulated_prompt)  # Получаем объект AIMessage
    reformulated_query = reformulated_query_result.content  # Доступ к содержимому
    
    # Шаг 2: Получаем документы с использованием RAG-Fusion и RRF
    docs = create_rag_fusion_retrieval_chain(retriever, reformulated_query)
    
    # Шаг 3: Генерируем ответ на основе контекста
    result = question_answer_chain.invoke({"input": query, "context": docs, "chat_history": chat_history})
    
    return result

# Функция для моделирования непрерывного чата
def continual_chat():
    print("Начните общение с ИИ помошником о компании Intellion! Введите ‘выход’, чтобы завершить разговор.")
    chat_history = []  # Хранение истории чата
    while True:
        query = input("ВЫ: ")  # Получение запроса от пользователя
        if query.lower() == "выход":  # Если пользователь хочет завершить чат
            break
        
        # Обрабатываем запрос пользователя через систему RAG с RRF
        result = process_query(query, chat_history)
        
        # Печать вопроса пользователя и ответа ИИ
        print(f"Вы: {query}")
        print(f"AI: {result}")  # Выводим результат, так как это строка
        
        # Обновление истории чата
        chat_history.append(HumanMessage(content=query))
        chat_history.append(SystemMessage(content=result))

# Запуск непрерывного чата
if __name__ == "__main__":
    continual_chat()

Начните общение с ИИ помошником о компании Intellion! Введите ‘выход’, чтобы завершить разговор.
Вы: Как ты можешь помочь?
AI: Я могу ответить на вопросы о компании Intelion и ее деятельности, а также предоставить информацию о людях, связанных с ней, если такая информация есть. Если у вас есть конкретный вопрос, задайте его, и я постараюсь помочь!
Вы: Можешь рассказать историю компании в 5 пунктах?
AI: 1. Компания Intelion была основана в 2017 году и изначально предоставляла услуги, связанные с вычислительными технологиями.
2. В 2023 году Intelion стала соучредителем инициативы по регулированию отрасли.
3. Компания зарекомендовала себя как промышленный оператор по продаже и обслуживанию вычислительного оборудования.
4. За последние 5 лет выручка Intelion увеличилась почти в 40 раз, что свидетельствует о значительном росте.
5. Для поддержания высоких темпов роста Intelion привлекает частные инвестиции в два направления.
Вы: А кто ген директор компании?
AI: Я не знаю, кто является генера